# Object Detection in Videos

In this tutorial, we'll demonstrate how to use Pixeltable to do frame-by-frame object detection, made simple through Pixeltable's video-related functionality:

- automatic frame extraction
- running complex functions against frames (in this case, the YOLOX object detection models)
- reassembling frames back into videos
  We'll be working with a single video file from Pixeltable's test data repository.

This tutorial assumes you're at least somewhat familiar with Pixeltable; a good place to learn more is the [Pixeltable Documentation](https://docs.pixeltable.com/overview/pixeltable).

<div class="alert alert-block alert-info"><!-- mdx:none -->
<b>If you are running this tutorial in Colab:</b>
In order to make the tutorial run a bit snappier, let's switch to a GPU-equipped instance for this Colab session. To do that, click on the <code>Runtime -> Change runtime type</code> menu item at the top, then select the <code>GPU</code> radio button and click on <code>Save</code>.
</div>

## Creating a tutorial directory and table

First, let's make sure the packages we need for this tutorial are installed: Pixeltable itself, PyTorch, and the YOLOX object detection library.

In [ ]:
%pip install -qU pixeltable pixeltable-yolox

All data in Pixeltable is stored in tables, which in turn reside in directories. We'll begin by creating a `detection_demo` directory and a table to hold our videos, with a single column of type `pxt.Video`.

In [ ]:
import pixeltable as pxt

pxt.create_dir('detection_demo', if_exists='replace_force')
videos_table = pxt.create_table(
    'detection_demo.videos',
    {'video': pxt.Video}
)

In order to interact with the frames, we take advantage of Pixeltable's component view concept: we create a "view" of our video table that contains one row for each frame of each video in the table. Pixeltable provides the built-in `frame_iterator` for this.

In [ ]:
from pixeltable.functions.video import frame_iterator

frames_view = pxt.create_view(
    'detection_demo.frames',
    videos_table,
    # `fps` determines the frame rate; a value of `0`
    # indicates the native frame rate of the video.
    iterator=frame_iterator(videos_table.video, fps=0)
)

You'll see that neither the `videos` table nor the `frames` view has any actual data yet, because we haven't yet added any videos to the table. However, the `frames` view is now configured to automatically track the `videos` table as new data shows up.

The new view is automatically configured with six columns:

- `pos` - a system column that is part of every component view
- `video` - the column inherited from our base table (all base table columns are visible in any of its views)
- `frame_idx`, `pos_msec`, `pos_frame`, `frame` - these four columns are created by the `frame_iterator`.

Let's have a look at the new view:

In [ ]:
frames_view

We'll now insert a single row into the videos table, containing a video of a busy intersection in Bangkok.

In [ ]:
videos_table.insert([
    {
        'video': 'https://raw.githubusercontent.com/pixeltable/pixeltable/release/docs/resources/bangkok.mp4'
    }
])

Notice that both the `videos` table and `frames` view were automatically updated, expanding the single video into 461 rows in the view. Let's have a look at `videos` first.

In [ ]:
videos_table.show()

Now let's peek at the first five rows of `frames`:

In [ ]:
frames_view.select(
    frames_view.pos,
    frames_view.frame,
    frames_view.frame.width,
    frames_view.frame.height
).show(5)

One advantage of using Pixeltable's component view mechanism is that Pixeltable does not physically store the frames. Instead, Pixeltable re-extracts the frames on retrieval using the frame index, which can be done very efficiently and avoids any storage overhead (which can be quite substantial for video frames).

## Object Detection with Pixeltable

Now let's apply an object detection model to our frames. Pixeltable includes built-in support for a number of models; we're going to use the YOLOX family of models, which are lightweight models with solid performance. We first import the `yolox` Pixeltable function.

In [ ]:
from pixeltable.functions.yolox import yolox

Pixeltable functions operate on columns and expressions using standard Python function call syntax. Here's an example that shows how we might experiment with applying one of the YOLOX models to the first few frames in our video, using Pixeltable's powerful `select` comprehension.

In [ ]:
# Show the results of applying the `yolox_tiny` model
# to the first few frames in the table.

frames_view.select(
    frames_view.frame,
    yolox(frames_view.frame, model_id='yolox_tiny')
).head(3)

It may appear that we just ran the YOLOX inference over the entire view of 461 frames, but remember that Pixeltable evaluates expressions lazily: in this case, it only ran inference over the 3 frames that we actually displayed.

The inference output looks like what we'd expect, so let's add a _computed column_ that runs inference over the entire view (computed columns are discussed in detail in the [Computed Columns](https://github.com/pixeltable/pixeltable/blob/release/docs/tutorials/computed-columns.ipynb) tutorial). Remember that once a computed column is created, Pixeltable will update it incrementally any time new rows are added to the view. This is a convenient way to incorporate inference (and other operations) into data workflows.

This _will_ cause Pixeltable to run inference over all 461 frames, so please be patient.

In [ ]:
# Create a computed column to compute detections using the `yolox_tiny`
# model.
# We'll adjust the confidence threshold down a bit (the default is 0.5)
# to pick up even more bounding boxes.

frames_view.add_computed_column(detections_tiny=yolox(
    frames_view.frame, model_id='yolox_tiny', threshold=0.25
))

The new column is now part of the schema of the `frames` view:

In [ ]:
frames_view

The data in the computed column is now stored for fast retrieval.

In [ ]:
frames_view.select(
    frames_view.frame,
    frames_view.detections_tiny
).show(3)

Now let's create a new set of images, in which we superimpose the detected bounding boxes on top of the original images. We'll use the handy built-in `draw_bounding_boxes` UDF for this. We could create a new computed column to hold the superimposed images, but we don't have to; sometimes it's easier just to use a `select` comprehension, as we did when we were first experimenting with the detection model.

In [ ]:
import pixeltable.functions as pxtf

frames_view.select(
    frames_view.frame,
    pxtf.vision.draw_bounding_boxes(
        frames_view.frame,
        frames_view.detections_tiny.bboxes,
        width=4
    )
).show(1)

Our `select` comprehension ranged over the entire table, but just as before, Pixeltable computes the output lazily: image operations are performed at retrieval time, so in this case, Pixeltable drew the annotations just for the one frame that we actually displayed.

Looking at individual frames gives us some idea of how well our detection algorithm works, but it would be more instructive to turn the visualization output back into a video.

We do that with the built-in function `make_video()`, which is an aggregation function that takes a frame index (actually: any expression that can be used to order the frames; a timestamp would also work) and an image, and then assembles the sequence of images into a video.

In [ ]:
frames_view.group_by(videos_table).select(
    pxt.functions.video.make_video(
        frames_view.pos,
        pxtf.vision.draw_bounding_boxes(
            frames_view.frame,
            frames_view.detections_tiny.bboxes,
            width=4
        )
    )
).show(1)

## Comparing Object Detection Models

The detections that we get out of `yolox_tiny` are passable, but a little choppy. Suppose we want to experiment with a more powerful object detection model, to see if there is any improvement in detection quality. We can create an additional column to hold the new inferences. The larger model takes longer to download and run, so please be patient.

In [ ]:
# Here we use the larger `yolox_m` (medium) model.

frames_view.add_computed_column(detections_m=yolox(
    frames_view.frame, model_id='yolox_m', threshold=0.25
))

Let's see the results of the two models side-by-side.

In [ ]:
frames_view.group_by(videos_table).select(
    pxt.functions.video.make_video(
        frames_view.pos,
        pxtf.vision.draw_bounding_boxes(
            frames_view.frame,
            frames_view.detections_tiny.bboxes,
            width=4
        )
    ),
    pxt.functions.video.make_video(
        frames_view.pos,
        pxtf.vision.draw_bounding_boxes(
            frames_view.frame,
            frames_view.detections_m.bboxes,
            width=4
        )
    )
).show(1)

Running the videos side-by-side, we can see that the larger model is higher in quality: less flickering, with more stable boxes from frame to frame.

## Evaluating Models Against a Ground Truth

In order to do a quantitative evaluation of model performance, we need a ground truth to compare them against. Let's generate some (synthetic) "ground truth" data by running against the largest YOLOX model available. It will take even longer to cache and evaluate this model.

In [ ]:
frames_view.add_computed_column(detections_x=yolox(
    frames_view.frame, model_id='yolox_x', threshold=0.25
))

Let's have a look at our enlarged view, now with three `detections` columns.

In [ ]:
frames_view

We're going to be evaluating the generated detections with the commonly-used [mean average precision](https://learnopencv.com/mean-average-precision-map-object-detection-model-evaluation-metric/) metric (mAP).

The mAP metric is based on per-frame metrics, such as true and false positives per detected class, which are then aggregated into a single (per-class) number. In Pixeltable, functionality is available via the `eval_detections()` and `mean_ap()` built-in functions.

In [ ]:
from pixeltable.functions.vision import eval_detections, mean_ap

frames_view.add_computed_column(eval_yolox_tiny=eval_detections(
    pred_bboxes=frames_view.detections_tiny.bboxes,
    pred_labels=frames_view.detections_tiny.labels,
    pred_scores=frames_view.detections_tiny.scores,
    gt_bboxes=frames_view.detections_x.bboxes,
    gt_labels=frames_view.detections_x.labels
))

frames_view.add_computed_column(eval_yolox_m=eval_detections(
    pred_bboxes=frames_view.detections_m.bboxes,
    pred_labels=frames_view.detections_m.labels,
    pred_scores=frames_view.detections_m.scores,
    gt_bboxes=frames_view.detections_x.bboxes,
    gt_labels=frames_view.detections_x.labels
))

Let's take a look at the output.

In [ ]:
frames_view.select(
    frames_view.eval_yolox_tiny,
    frames_view.eval_yolox_m
).show(1)

The computation of the mAP metric is now simply a query over the evaluation output, aggregated with the `mean_ap()` function.

In [ ]:
frames_view.select(
    mean_ap(frames_view.eval_yolox_tiny),
    mean_ap(frames_view.eval_yolox_m)
).show()

This two-step process allows you to compute mAP at every granularity: over your entire dataset, only for specific videos, only for videos that pass a certain filter, etc. Moreover, you can compute this metric any time, not just during training, and use it to guide your understanding of your dataset and how it affects the quality of your models.